# Business Case 
This code represents a comprehensive web scraping and data analysis pipeline for extracting and analyzing bicycle product information from online listings. 

By automating the process of fetching details such as price, state, size, and type from multiple web pages, businesses involved in the retail or e-commerce sector can utilize this pipeline to gather competitive pricing insights, understand market trends, and streamline inventory management. 

Additionally, the generated aggregated data on average prices per bicycle type and state can inform pricing strategies, inventory assortment decisions, and geographical targeting, enabling businesses to optimize their product offerings and pricing models to meet customer demand effectively.

In [65]:
# Importing necessary modules
from selenium import webdriver  # Module for browser automation
from selenium.webdriver.common.by import By  # Module for locating elements by different strategies
from selenium.webdriver.common.keys import Keys  # Module for keyboard keys simulation
from bs4 import BeautifulSoup  # Module for HTML parsing
import math # Module for mathematical functions (lambda x)
import time # Module for time-related functions (sleep time for web scrapping to prevent from account blockage)
import pandas as pd
from selenium.webdriver.common.action_chains import ActionChains
import requests
import re
import numpy as np

In [66]:
driver = webdriver.Chrome()#executable_path=r'C:\Users\amosw\Downloads\chromedriver-win64\chromedriver-win64\chromedriver.exe')

# Open a webpage
url='https://es.wallapop.com/app/search?'
driver.get(url)

In [67]:
# Pause the program execution for 2 seconds
time.sleep(2)

# Locate an HTML element by its ID ('onetrust-reject-all-handler') and simulate a click on it (Reject Cookies)
driver.find_element(By.ID, value='onetrust-reject-all-handler').click()

In [68]:
def css_selector(x):
    # Concatenate a period ('.') to the string resulting from joining the split elements by periods ('.') to create css selector
    return '.' + '.'.join(x.split(' '))

In [69]:
#category filter
# Wait for 3 seconds for the page to load
time.sleep(3)

# Find the elements matching the CSS selector generated by the 'css_selector' function
# Click on the second element found by the CSS selector to open the category filter
driver.find_elements(By.CSS_SELECTOR, css_selector('Bubble__content_wrapper px-3 d-flex justify-content-center align-items-center'))[1].click()

In [70]:
# bicycle category
# Wait for 3 seconds for the page to load
time.sleep(3)

# Find elements matching the CSS selector generated by the 'css_selector' function
# Click on the 11th (index 10, as indexing starts from 0) element found by the CSS selector which corresponds to the bicycle filter
driver.find_elements(By.CSS_SELECTOR, css_selector('GridSelectOption p-2 d-flex flex-column justify-content-center align-items-center'))[10].click()


In [71]:
#location filter
# Wait for 3 seconds for the page to load
time.sleep(3)

# Find elements matching the CSS selector generated by the 'css_selector' function
# Click on the second element found by the CSS selector to open the location filter
driver.find_elements(By.CSS_SELECTOR, css_selector('Bubble px-3 d-flex justify-content-center align-items-center Bubble--active'))[1].click()

In [72]:
# Pause the program execution for 3 seconds
time.sleep(3)

# Click on the element for location filter input
driver.find_element(By.CSS_SELECTOR, css_selector('LocationFilter__input py-0 px-5 w-100 form-control ng-untouched ng-pristine ng-valid')).click()

# Clear the content of said element
driver.find_element(By.CSS_SELECTOR, css_selector('LocationFilter__input py-0 px-5 w-100 form-control ng-untouched ng-pristine ng-valid')).clear()

# Input the text 'Barcelona' and simulate pressing the ENTER key
driver.find_element(By.CSS_SELECTOR, css_selector('LocationFilter__input py-0 px-5 w-100 form-control ng-pristine ng-valid ng-touched')).send_keys('Barcelona' + Keys.ENTER)

In [73]:
# select location option
# Wait for 5 seconds to allow time for the selection option (Barcelona) to appear
time.sleep(5)

# Simulate pressing the ENTER key to select the option
driver.find_element(By.CSS_SELECTOR, css_selector("LocationFilter__input py-0 px-5 w-100 form-control ng-valid ng-touched ng-dirty open")).send_keys(Keys.ENTER)

In [74]:
# select distance limit of 10km
# Pause the program execution for 5 seconds
time.sleep(5)

# Find the slider element using the generated CSS selector for the slider
slider = driver.find_elements(By.CSS_SELECTOR, css_selector('ngx-slider-span ngx-slider-pointer ngx-slider-pointer-min'))[0]

# Iterate five times to perform the action of moving the slider left using the ARROW_LEFT key
for i in range(5):
    slider.send_keys(Keys.ARROW_LEFT)


In [75]:
# accept location filter
# Wait for 5 seconds to ensure the location filter is ready
time.sleep(5)

# Find and click on an element representing the acceptance of the location filter
driver.find_element(By.CSS_SELECTOR, css_selector('btn btn-filter btn-primary')).click()


In [76]:
# price filter
# Wait for 3 seconds to ensure that the page is loaded
time.sleep(3)

# Find elements matching the CSS selector generated by the 'css_selector' function
# Click on the second element found by the CSS selector related to the price filter
driver.find_elements(By.CSS_SELECTOR, css_selector('Bubble px-3 d-flex justify-content-center align-items-center Bubble--active'))[1].click()


In [77]:
def number_clicks(x): # to find the wanted position of the price option
    # Initialize an empty list to store options
    list_options = []
    
    # Populate the list with specific ranges of values representing options
    list_options = list_options + list(range(0, 51)) + list(range(50, 101, 5)) + list(range(100, 301, 10)) + list(range(300, 501, 50)) + list(range(500, 1001, 100)) + list(range(1000, 3001, 250)) + list(range(3000, 5001, 500)) + list(range(5000, 10001, 1000)) + list(range(10000, 18751, 1250))

    # Return the index in the list that corresponds to the value closest to 'x'
    return min(range(len(list_options)), key=lambda i: abs(list_options[i] - x))


In [78]:
# Minimum Price
# Pause the program execution for 5 seconds
time.sleep(5)

# Find the slider element related to the minimum price using the generated CSS selector
slider = driver.find_elements(By.CSS_SELECTOR, css_selector('ngx-slider-span ngx-slider-pointer ngx-slider-pointer-min'))[1]

# Adjust the slider to set the minimum price by moving it right based on the calculated number of clicks
for i in range(number_clicks(300)): # in this case we put 300€ as the minimum for time reasons, but this can be freely changed 
    slider.send_keys(Keys.ARROW_RIGHT)

# Find the slider element related to the maximum price using the generated CSS selector
slider = driver.find_elements(By.CSS_SELECTOR, css_selector('ngx-slider-span ngx-slider-pointer ngx-slider-pointer-max'))[1]

# Adjust the slider to set the maximum price by moving it left based on the calculated number of clicks
for i in range(122 - number_clicks(350)): # the 350€ option was chosen to reduce the number of results
    slider.send_keys(Keys.ARROW_LEFT)

# Pause the program execution for 3 seconds
time.sleep(3)

# Click on the second filter button found using the generated CSS selector for applying the price filter
driver.find_elements(By.CSS_SELECTOR, css_selector('btn btn-filter btn-primary'))[1].click()


In [79]:
# Subcategory Filter
# Wait for 3 seconds to ensure the page is loaded
time.sleep(3)
# open subcategory filter
driver.find_elements(By.CSS_SELECTOR, css_selector('Bubble px-3 d-flex justify-content-center align-items-center Bubble--active'))[1].click()


time.sleep(3)
# open the bicycles and tricycles option
driver.find_elements(By.CSS_SELECTOR, css_selector('MultiSelectOption__container w-100 align-items-center justify-content-between'))[1].click()

# Click on specific checkboxes: Bicicletas de carretera, Bicicletas plegables, MTB
driver.find_elements(By.CSS_SELECTOR, css_selector('Checkbox__mark d-block position-relative m-0'))[2].click()
driver.find_elements(By.CSS_SELECTOR, css_selector('Checkbox__mark d-block position-relative m-0'))[4].click()
driver.find_elements(By.CSS_SELECTOR, css_selector('Checkbox__mark d-block position-relative m-0'))[6].click()

# Wait for 3 seconds and then click on apply
time.sleep(3)
driver.find_elements(By.CSS_SELECTOR, css_selector('btn btn-filter btn-primary'))[2].click()

In [80]:
# Product State Filter
# Wait for 3 seconds to ensure the page is loaded
time.sleep(3)
# open product state filter
driver.find_elements(By.CSS_SELECTOR, css_selector('Bubble px-3 d-flex justify-content-center align-items-center Bubble--active'))[1].click()
time.sleep(3)

# Select checkboxes representing product states: Nuevo, Como nuevo, En bueno estado
driver.find_elements(By.CSS_SELECTOR, css_selector('Checkbox__mark d-block position-relative m-0'))[3].click()
driver.find_elements(By.CSS_SELECTOR, css_selector('Checkbox__mark d-block position-relative m-0'))[4].click()
driver.find_elements(By.CSS_SELECTOR, css_selector('Checkbox__mark d-block position-relative m-0'))[5].click()
time.sleep(3)

# Click on the sixth filter button found using the generated CSS selector for product state filtering
driver.find_elements(By.CSS_SELECTOR, css_selector('btn btn-filter btn-primary'))[5].click()

In [81]:
# Pause the program execution for 10 seconds
time.sleep(10)

# Click on 'Ver mas productos' button to show more results
driver.find_elements(By.CSS_SELECTOR, css_selector('btn btn-primary'))[-1].click()

In [82]:
# run the code every 8 seconds to fully load results
SCROLL_PAUSE_TIME = 8

# Get scroll height
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        # once reached the bottom it stops
        break
    last_height = new_height
    

In [83]:
# Create a BeautifulSoup object ('soup') by parsing the HTML content of the current web page using 'html.parser'
soup=BeautifulSoup(driver.page_source, 'html.parser')

In [84]:
# Find all <a> elements with the class 'ItemCardList__item ng-star-inserted' using BeautifulSoup which are the bicycles' posts
bicycles = soup.find_all('a', class_='ItemCardList__item ng-star-inserted')

# Create an empty list 'posts' to store extracted information
posts = []

# Iterate through each bicycle element
for bike in bicycles:
    # Extract specific information such as title, post link, picture link, and price
    title = bike['title']
    link_post = bike['href']
    link_picture = bike.find('img')['src']
    price = float(bike.find('span', class_='ItemCard__price ItemCard__price--bold').text.strip()[:-1])
    
    # Append the extracted information as a list to the 'posts' list
    posts.append([title, link_post, link_picture, price])

# Create a pandas DataFrame ('df') using the extracted posts information and define column names
df = pd.DataFrame(data=posts, columns=['Title', 'Link', 'Picture', 'Price'])

# Display the first few rows of the DataFrame 'df'
df.head()


,Title,Link,Picture,Price
0,Bicicleta Merida carretera road lite 905,https://es.wallapop.com/item/bicicleta-merida-...,https://cdn.wallapop.com/images/10420/fs/fa/__...,350.0
1,Bicicleta Scott,https://es.wallapop.com/item/bicicleta-scott-9...,https://cdn.wallapop.com/images/10420/fn/q4/__...,350.0
2,BICI TREK FUEL 80 doble suspension,https://es.wallapop.com/item/bici-trek-fuel-80...,https://cdn.wallapop.com/images/10420/ed/1h/__...,350.0
3,Cannondale Prefin Subida,https://es.wallapop.com/item/cannondale-prefin...,https://cdn.wallapop.com/images/10420/e5/kw/__...,300.0
4,"Btt nueva INDI MTB 27,5” disco mec. 21v NEGRO",https://es.wallapop.com/item/btt-nueva-indi-mt...,https://cdn.wallapop.com/images/10420/ck/zx/__...,350.0


In [85]:
# Remove duplicate rows from the DataFrame 'df'
df=df.drop_duplicates()

In [86]:
# Iterate through each row in the DataFrame 'df'
for i, bike in df.iterrows():
    # Fetch the webpage content from the URL specified in the 'Link' column
    response = requests.get(bike['Link'])
    print(i)  # Print the current index as an indication of progress
    soup = BeautifulSoup(response.text, 'html.parser')  # Create a BeautifulSoup object from the response text
    
    # Try to find and extract the 'State' element from the webpage
    try:
        state_element = soup.find('span', class_='item-detail-additional-specifications_ItemDetailAdditionalSpecifications__text__ByuCm').text.strip()
    except:
        state_element = np.NaN

    # Update the 'State' column in the DataFrame with the extracted state information
    df.at[i, 'State'] = state_element

    # Try to find and extract the 'Description' section from the webpage
    try:
        description_section = soup.find('section', class_='item-detail_ItemDetail__description__7rXXT py-4').text.strip()
    except:
        description_section = np.NaN

    # Update the 'Description' column in the DataFrame with the extracted description information
    df.at[i, 'Description'] = description_section
    
    # Define a list of words to check for kid-related bike descriptions or titles
    words_to_check = ["niño/a", "niño", "niña", "niños", "niñas", "niño/as"]
    
    # Check if the bike description or title contains words related to kids
    kid_bike = (any(word in description_section for word in words_to_check) or any(word in bike['Title'] for word in words_to_check))
    
    # Update the 'Kid' column in the DataFrame based on the presence of kid-related words
    df.at[i, 'Kid'] = kid_bike
    
    # Search for the size information using regular expression in the description section
    match = re.search(r'talla (\d{2}|\b(?:s|m|l)\b)', description_section.lower())
    
    # Extract and update 'Size' and 'Size (letter)' columns based on the matched size information
    if match:
        size_value = match.group(1)
        
        if size_value.isdigit():
            size = int(size_value)
            size_letter = np.NaN
        else:
            size = np.NaN
            size_letter = size_value.upper()
    else:
        size = np.NaN
        size_letter = np.NaN
    
    df.at[i, 'Size'] = size
    df.at[i, 'Size (letter)'] = size_letter
    
    # Try to find and extract the 'Type' of the bike from the webpage
    try:
        type_bike = soup.find_all('div', 'item-detail-taxonomies_ItemDetailTaxonomies__bubbles__yO53X')[1].span.text
    except:
        type_bike = np.NaN

    # Update the 'Type' column in the DataFrame with the extracted bike type
    df.at[i, 'Type'] = type_bike
    
    # Try to find and extract the 'Edited' information from the webpage
    try:
        edited = soup.find('span', class_='item-detail-stats_ItemDetailStats__description__vjz96 text-truncate mr-2').text.strip()
    except:
        edited = np.NaN
    
    # Update the 'Edited' column in the DataFrame with the extracted edited information
    df.at[i, 'Edited'] = edited

# Display the first few rows of the DataFrame 'df'
df.head()


0


C:\Users\amosw\AppData\Local\Temp\ipykernel_25832\3561619752.py:15: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Buen estado' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[i, 'State'] = state_element
C:\Users\amosw\AppData\Local\Temp\ipykernel_25832\3561619752.py:24: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Bicicleta Merida de Carretera en buen estado ideal para iniciación.
Talla M. Frenos de pinza. 9 velocidades 11-28 y 50 x 34. 
Grupo shimano ultegra.' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[i, 'Description'] = description_section
C:\Users\amosw\AppData\Local\Temp\ipykernel_25832\3561619752.py:33: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'False' has dtype 

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52


,Title,Link,Picture,Price,State,Description,Kid,Size,Size (letter),Type,Edited
0,Bicicleta Merida carretera road lite 905,https://es.wallapop.com/item/bicicleta-merida-...,https://cdn.wallapop.com/images/10420/fs/fa/__...,350.0,Buen estado,Bicicleta Merida de Carretera en buen estado i...,False,NaN,M,Bicicletas de carretera,Editado hace menos de 1 mes
1,Bicicleta Scott,https://es.wallapop.com/item/bicicleta-scott-9...,https://cdn.wallapop.com/images/10420/fn/q4/__...,350.0,Buen estado,Muy cuidada,False,NaN,NaN,MTB,Editado hace 2 horas
2,BICI TREK FUEL 80 doble suspension,https://es.wallapop.com/item/bici-trek-fuel-80...,https://cdn.wallapop.com/images/10420/ed/1h/__...,350.0,Buen estado,Bicicleta Mountain Bike doble suspensión Trek ...,False,NaN,S,MTB,Editado hace menos de 1 mes
3,Cannondale Prefin Subida,https://es.wallapop.com/item/cannondale-prefin...,https://cdn.wallapop.com/images/10420/e5/kw/__...,300.0,Buen estado,"Bicicleta Cannondale, la mayoria de piesas son...",False,NaN,NaN,Bicicletas de carretera,Editado hace menos de 1 mes
4,"Btt nueva INDI MTB 27,5” disco mec. 21v NEGRO",https://es.wallapop.com/item/btt-nueva-indi-mt...,https://cdn.wallapop.com/images/10420/ck/zx/__...,350.0,Nuevo,Bicicleta de montaña nueva con garantia \nRued...,False,NaN,NaN,MTB,Editado hace menos de 1 mes


In [87]:
# Create a new DataFrame 'agg' by grouping columns 'Price', 'Type', and 'State', and calculating the mean price for each unique combination of 'Type' and 'State'
agg = df[['Price', 'Type', 'State']].groupby(['Type', 'State']).mean()

# Display the first few rows of the aggregated DataFrame 'agg'
agg.head()

Price
Type                    State                  
Bicicletas de carretera Buen estado  333.181818
                        Como nuevo   344.000000
Bicicletas plegables    Como nuevo   334.750000
                        Nuevo        315.000000
MTB                     Buen estado  320.764706

In [88]:
df.to_csv('data.csv', index=False) # save dataframe